In [6]:
import json

with open("../dicease_revised.json", "r", encoding="utf-8") as file:
    disease_data = json.load(file)
def find_disease_and_sons(data, target_disease):
    """递归查找目标疾病及其所有子节点"""
    for disease_name, disease_info in data.items():
        if disease_name == target_disease:
            # 找到目标疾病，开始递归获取子节点
            return [disease_name] + get_all_sons(disease_info.get("包含", {}))
        elif "包含" in disease_info:
            # 继续在子疾病中查找
            result = find_disease_and_sons(disease_info["包含"], target_disease)
            if result:
                return result
    return []

def get_all_sons(data):
    """递归获取所有子节点"""
    sons = []
    for sub_disease, sub_info in data.items():
        sons.append(sub_disease)
        if "包含" in sub_info:
            sons.extend(get_all_sons(sub_info["包含"]))
    return sons

def create_appointment_file(target_disease, department_name):
    """生成就诊.txt文件"""
    # 查找目标疾病及其所有子节点
    all_diseases = find_disease_and_sons(disease_data, target_disease)
    
    if not all_diseases:
        print(f"未找到疾病：{target_disease}")
        return
    print(f"找到疾病：{target_disease}，共{len(all_diseases)}个子疾病")
    # 写入文件
    with open("治疗疾病.txt", "a", encoding="utf-8") as file:
        for disease in all_diseases:
            file.write(f"{department_name} {disease}\n")

# 示例调用
disease_name = "牙周病"
department_name = "松牙固定术"
create_appointment_file(disease_name, department_name)

找到疾病：牙周病，共34个子疾病


# 关系保存至图数据库

In [11]:
import os
from py2neo import Graph, NodeMatcher, Relationship, Node
class Neo4jRelationStorage:
    """
    用于将关系存储到 Neo4j 数据库的工具类。
    """
    def __init__(self, url, username, password):
        """
        初始化 Neo4j 数据库连接。
        
        :param url: Neo4j 数据库连接地址
        :param username: 数据库用户名
        :param password: 数据库密码
        """
        self.graph = Graph(url, auth=(username, password))
        self.node_matcher = NodeMatcher(self.graph)

    def find_or_create_node(self, label, property_key, property_value):
        """
        查找或创建指定标签和属性值的节点。
        
        :param label: 节点标签
        :param property_key: 属性键
        :param property_value: 属性值
        :return: 找到或创建的节点
        """
        query = f"MATCH (n:{label}) WHERE n.{property_key} = $value RETURN n"
        node = self.graph.run(query, value=property_value).data()

        if not node:
            return None

        return node[0]['n']

    def store_relations(self, relation_list, node1_label, node2_label, relation_type, property_key):
        """
        将关系列表存储到 Neo4j 数据库中。
        
        :param relation_list: 关系列表，格式为 [[节点1名称, 节点2名称], ...]
        :param node1_label: 节点1的标签
        :param node2_label: 节点2的标签
        :param relation_type: 关系类型（字符串）
        """
        for [node1_name, node2_name] in relation_list:
            # 查找或创建节点1
            node1 = self.find_or_create_node(node1_label, property_key, node1_name)
            if not node1:
                print(f"节点1（标签：{node1_label}，名称：{node1_name}）不存在")
                continue
            # 查找或创建节点2
            node2 = self.find_or_create_node(node2_label, property_key, node2_name)
            if not node2:
                print(f"节点2（标签：{node2_label}，名称：{node2_name}）不存在")
                continue
            
            # 创建或更新关系
            relationship = Relationship(node1, relation_type, node2)
            self.graph.merge(relationship)  # 使用 merge 确保不会重复创建关系
        
        print(f"关系数据（节点1标签：{node1_label}，节点2标签：{node2_label}，关系类型：{relation_type}）已成功保存到 Neo4j")

def process_files(directory, neo4j_storage):
    """
    处理目录下的所有关系文件，并将它们保存到 Neo4j 数据库中。
    
    :param directory: 包含关系文件的目录路径
    :param neo4j_storage: Neo4jRelationStorage 实例
    """
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                data = file.read()
            entity_list = data.split("\n")
            relation_list = []
            for entity in entity_list:
                if entity:
                    item=entity.split(" ")
                    if len(item)!=2:
                        print(f"文件{filename}中的数据格式不正确：{entity}")
                        continue
                    relation_list.append(entity.split(" "))
            
            # 根据文件名确定节点标签和关系类型
            if "就诊" in filename:
                node1_label = "口腔疾病"
                node2_label = "科室"
                relation_type = "就诊"
            elif "包含组成" in filename:
                node1_label = "器械"
                node2_label = "器械"
                relation_type = "包含组成"
            elif "检查使用器械" in filename:
                node1_label = "检查"
                node2_label = "器械"
                relation_type = "使用器械"
            elif "治疗使用器械llm" in filename:
                node1_label = "治疗"
                node2_label = "器械"
                relation_type = "使用器械"
            elif "治疗使用药物llm" in filename:
                node1_label = "治疗"
                node2_label = "药物"
                relation_type = "使用药物"
            elif "治疗疾病llm" in filename:
                node1_label = "治疗"
                node2_label = "口腔疾病"
                relation_type = "治疗"
            elif "预防使用器械" in filename:
                node1_label = "预防"
                node2_label = "器械"
                relation_type = "使用器械"
            elif "预防使用药物" in filename:
                node1_label = "预防"
                node2_label = "药物"
                relation_type = "使用药物"
            elif "预防疾病" in filename:
                node1_label = "预防"
                node2_label = "口腔疾病"
                relation_type = "预防"
            else:
                print(f"未知文件类型：{filename}")
                continue
            
            neo4j_storage.store_relations(relation_list, node1_label, node2_label, relation_type, "名称")

# 示例调用
if __name__ == "__main__":
    # 初始化 Neo4j 存储工具
    neo4j_storage = Neo4jRelationStorage(
        url="bolt://localhost:9687",
        username="neo4j",
        password="medical_neo4j"
    )
    
    # 处理关系文件
    process_files("./", neo4j_storage)

关系数据（节点1标签：预防，节点2标签：药物，关系类型：使用药物）已成功保存到 Neo4j
关系数据（节点1标签：治疗，节点2标签：口腔疾病，关系类型：治疗）已成功保存到 Neo4j
关系数据（节点1标签：预防，节点2标签：口腔疾病，关系类型：预防）已成功保存到 Neo4j
未知文件类型：治疗疾病.txt
关系数据（节点1标签：器械，节点2标签：器械，关系类型：包含组成）已成功保存到 Neo4j
未知文件类型：治疗使用器械.txt
未知文件类型：治疗使用药物.txt


# 大模型自动生成关系头尾节点

## 功能描述

本脚本用于自动化生成两个JSON文件之间的指定关系。通过调用大模型（如阿里云的DashScope API），脚本会遍历头实体文件和尾实体文件中的所有节点，判断是否存在指定的关系，并将符合条件的头尾节点对输出到指定的TXT文件中。

---

## 输入

1. **头实体JSON文件**：
   - 文件路径：`--head` 参数指定。
   - 文件内容：包含头实体的分类树图谱，JSON格式。
   - 示例：
     ```json
     {
         "疾病A": {
             "定义": "某种口腔疾病",
             "症状": "疼痛、肿胀"
         },
         "疾病B": {
             "定义": "另一种口腔疾病",
             "症状": "出血、溃疡"
         }
     }
     ```

2. **尾实体JSON文件**：
   - 文件路径：`--tail` 参数指定。
   - 文件内容：包含尾实体的分类树图谱，JSON格式。
   - 示例：
     ```json
     {
         "治疗方案A": {
             "定义": "某种治疗方案",
             "适用疾病": "疾病A"
         },
         "治疗方案B": {
             "定义": "另一种治疗方案",
             "适用疾病": "疾病B"
         }
     }
     ```

3. **关系名称**：
   - 通过 `--relation` 参数指定。
   - 示例：`治疗`。

4. **关系定义**：
   - 通过 `--definition` 参数指定。
   - 示例：`该治疗方案用于治疗该疾病`。

---

## 输出

1. **输出文件**：
   - 文件路径：`--output` 参数指定。
   - 文件格式：TXT文件，每行包含一对符合关系的头尾节点名称，用空格分隔。
   - 示例：
     ```
     疾病A 治疗方案A
     疾病B 治疗方案B
     ```

---

## 核心功能

1. **实体遍历**：
   - 递归遍历头实体和尾实体JSON文件中的所有节点，提取实体名称及其属性。

2. **关系判断**：
   - 动态生成提示词，调用大模型判断头实体和尾实体之间是否存在指定关系。
   - 提示词示例：
     ```
     请严格根据以下医学知识判断两个实体间是否存在“治疗”关系。
     关系定义：该治疗方案用于治疗该疾病。
     头实体名称：疾病A
     头实体属性：定义: 某种口腔疾病；症状: 疼痛、肿胀
     尾实体名称：治疗方案A
     尾实体属性：定义: 某种治疗方案；适用疾病: 疾病A
     答案只能是“是”或“否”，无需解释。
     ```

3. **结果保存**：
   - 将符合关系的头尾节点对保存到指定的TXT文件中。

---

## 使用示例

### 命令行运行

```bash
python generate_relations.py \
    --head disease.json \
    --tail treatment.json \
    --relation 治疗 \
    --definition "该治疗方案用于治疗该疾病" \
    --output disease_treatment_pairs.txt
```

In [19]:
import json
import dashscope
from tqdm import tqdm

dashscope.api_key = "sk-969f4200d53442a2a1733d1c0b1fb330"

def call_large_model(prompt):
    messages = [
        {"role": "system", "content": "你是一个医学知识图谱构建专家。"},
        {"role": "user", "content": prompt}
    ]
    try:
        response = dashscope.Generation.call(
            model="qwen-plus",
            messages=messages,
            result_format='message'
        )
        response_text = response['output']['choices'][0]['message']['content'].strip()
        return '是' if '是' in response_text else '否'
    except Exception as e:
        print(f"调用模型失败: {e}")
        return '否'

def get_json_block(data):
    json_block = {}
    for key, value in data.items():
        saved_info = {k: v for k, v in value.items() if k != '包含'}
        json_block[key] = saved_info
        if '包含' in value:
            child_block = get_json_block(value['包含'])
            json_block.update(child_block)
    return json_block

def format_entity_info(info):
    parts = []
    for k, v in info.items():
        if isinstance(v, dict):
            nested = ', '.join([f"{sub_k}: {sub_v}" for sub_k, sub_v in v.items()])
            parts.append(f"{k}: {nested}")
        elif isinstance(v, list):
            parts.append(f"{k}: {', '.join(map(str, v))}")
        else:
            parts.append(f"{k}: {v}")
    return '； '.join(parts)

def generate_prompt(head_entity, head_info, tail_entity, tail_info, relation_name, relation_definition):
    head_info_str = format_entity_info(head_info)
    tail_info_str = format_entity_info(tail_info)
    return f"""请严格根据以下医学知识判断两个实体间是否存在“{relation_name}”关系。
关系定义：{relation_definition}
头实体名称：{head_entity}
头实体属性：{head_info_str}
尾实体名称：{tail_entity}
尾实体属性：{tail_info_str}
答案只能是“是”或“否”，无需解释。"""

def find_relations(head_file, tail_file, relation_name, relation_definition, output_file):
    with open(head_file, 'r', encoding='utf-8') as f:
        head_entities = get_json_block(json.load(f))
    with open(tail_file, 'r', encoding='utf-8') as f:
        tail_entities = get_json_block(json.load(f))

    with open(output_file, 'w', encoding='utf-8') as out_f:
        total = len(head_entities) * len(tail_entities)
        progress_bar = tqdm(total=total, desc="处理进度", unit="pair")
        for head_name, head_info in head_entities.items():
            for tail_name, tail_info in tail_entities.items():
                prompt = generate_prompt(head_name, head_info, tail_name, tail_info, relation_name, relation_definition)
                response = call_large_model(prompt)
                if response == '是':
                    out_f.write(f"{head_name} {tail_name}\n")
                progress_bar.update(1)
        progress_bar.close()

if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser(description='自动生成知识图谱关系')
    parser.add_argument('--head', required=True, help='头实体JSON文件路径')
    parser.add_argument('--tail', required=True, help='尾实体JSON文件路径')
    parser.add_argument('--relation', required=True, help='关系名称（如治疗）')
    parser.add_argument('--definition', required=True, help='关系定义（如该治疗方案用于治疗该疾病）')
    parser.add_argument('--output', required=True, help='输出文件路径')
    args = parser.parse_args()
    
    find_relations(args.head, args.tail, args.relation, args.definition, args.output)

# 大模型自动生成关系头尾节点（基于专业知识的剪枝策略）

## 功能描述

本脚本在原有功能的基础上，改进了剪枝策略。通过调用大模型，结合口腔领域的专业知识和医学常识，动态判断是否需要剪枝。如果遇到模糊情况，则不剪枝，确保不会遗漏潜在的关系。

---

## 改进点

1. **基于专业知识的剪枝**：
   - 在遍历分类树时，调用大模型判断当前头实体和尾实体是否可能存在指定关系。
   - 如果大模型明确判断为“否”，则剪枝；否则继续遍历。

2. **模糊情况不剪枝**：
   - 如果大模型无法明确判断（例如返回模糊结果），则不剪枝，确保不会遗漏潜在的关系。

3. **效率与准确性平衡**：
   - 通过动态剪枝策略，在保证准确性的前提下，尽可能减少不必要的遍历和模型调用。

---

## 输入

与之前相同，包括：
1. 头实体JSON文件（`--head`）。
2. 尾实体JSON文件（`--tail`）。
3. 关系名称（`--relation`）。
4. 关系定义（`--definition`）。

---

## 输出

与之前相同，输出符合关系的头尾节点对到指定的TXT文件中。

---

## 核心功能

### 1. **基于专业知识的剪枝**
   - 在遍历分类树时，调用大模型判断当前头实体和尾实体是否可能存在指定关系。
   - 示例：
     - 头实体：`拔牙技术`
     - 尾实体：`牙周炎`
     - 大模型判断：`是`（可能存在“治疗”关系）。
     - 结果：不剪枝，继续遍历。

     - 头实体：`拔牙技术`
     - 尾实体：`颞下颌关节紊乱病`
     - 大模型判断：`否`（不可能存在“治疗”关系）。
     - 结果：剪枝，跳过该子树。

### 2. **模糊情况不剪枝**
   - 如果大模型返回模糊结果（例如“不确定”），则不剪枝，确保不会遗漏潜在的关系。

### 3. **关系判断**
   - 动态生成提示词，调用大模型判断头实体和尾实体之间是否存在指定关系。
   - 提示词示例：
     ```
     请严格根据以下医学知识判断两个实体间是否存在“治疗”关系。
     关系定义：该治疗方案用于治疗该疾病。
     头实体名称：拔牙技术
     头实体属性：定义: 通过手术拔除牙齿；适用疾病: 严重龋齿、牙周炎
     尾实体名称：牙周炎
     尾实体属性：定义: 牙龈和牙周组织的炎症；症状: 牙龈出血、牙齿松动
     答案只能是“是”或“否”，无需解释。
     ```

### 4. **结果保存**
   - 将符合关系的头尾节点对保存到指定的TXT文件中。

In [ ]:
import json
import dashscope
from tqdm import tqdm

dashscope.api_key = "sk-969f4200d53442a2a1733d1c0b1fb330"

def call_large_model(prompt):
    messages = [
        {"role": "system", "content": "你是一个医学知识图谱构建专家。"},
        {"role": "user", "content": prompt}
    ]
    try:
        response = dashscope.Generation.call(
            model="qwen-plus",
            messages=messages,
            result_format='message'
        )
        response_text = response['output']['choices'][0]['message']['content'].strip()
        return response_text
    except Exception as e:
        print(f"调用模型失败: {e}")
        return '不确定'

def format_entity_info(info):
    parts = []
    for k, v in info.items():
        if isinstance(v, dict):
            nested = ', '.join([f"{sub_k}: {sub_v}" for sub_k, sub_v in v.items()])
            parts.append(f"{k}: {nested}")
        elif isinstance(v, list):
            parts.append(f"{k}: {', '.join(map(str, v))}")
        else:
            parts.append(f"{k}: {v}")
    return '； '.join(parts)

def generate_prompt(head_entity, head_info, tail_entity, tail_info, relation_name, relation_definition):
    head_info_str = format_entity_info(head_info)
    tail_info_str = format_entity_info(tail_info)
    return f"""请严格根据以下医学知识判断两个实体间是否存在“{relation_name}”关系。
关系定义：{relation_definition}
头实体名称：{head_entity}
头实体属性：{head_info_str}
尾实体名称：{tail_entity}
尾实体属性：{tail_info_str}
答案只能是“是”或“否”，无需解释。"""

def should_prune(head_entity, head_info, tail_entity, tail_info, relation_name, relation_definition):
    """
    判断是否需要剪枝。
    - 调用大模型判断当前头实体和尾实体是否可能存在指定关系。
    - 如果大模型明确返回“否”，则剪枝；否则不剪枝。
    """
    prompt = generate_prompt(head_entity, head_info, tail_entity, tail_info, relation_name, relation_definition)
    response = call_large_model(prompt)
    return response == '否'
def get_json_block(data):
    json_block = {}
    for key, value in data.items():
        saved_info = {k: v for k, v in value.items() if k != '包含'}
        json_block[key] = saved_info
        if '包含' in value:
            child_block = get_json_block(value['包含'])
            json_block.update(child_block)
    return json_block
def traverse_tree(data, tail_entities, relation_name, relation_definition, output_file, progress_bar):
    for entity_name, entity_info in data.items():
        saved_info = {k: v for k, v in entity_info.items() if k != '包含'}
        for tail_name, tail_info in tail_entities.items():
            if should_prune(entity_name, saved_info, tail_name, tail_info, relation_name, relation_definition):
                continue  # 剪枝
            prompt = generate_prompt(entity_name, saved_info, tail_name, tail_info, relation_name, relation_definition)
            response = call_large_model(prompt)
            if response == '是':
                output_file.write(f"{entity_name} {tail_name}\n")
            progress_bar.update(1)
        if '包含' in entity_info:
            traverse_tree(entity_info['包含'], tail_entities, relation_name, relation_definition, output_file, progress_bar)

def find_relations(head_file, tail_file, relation_name, relation_definition, output_file):
    with open(head_file, 'r', encoding='utf-8') as f:
        head_data = json.load(f)
    with open(tail_file, 'r', encoding='utf-8') as f:
        tail_entities = get_json_block(json.load(f))

    total = sum(1 for _ in traverse_tree(head_data, tail_entities, None, None, None, None))
    with open(output_file, 'w', encoding='utf-8') as out_f:
        progress_bar = tqdm(total=total, desc="处理进度", unit="pair")
        traverse_tree(head_data, tail_entities, relation_name, relation_definition, out_f, progress_bar)
        progress_bar.close()

if __name__ == '__main__':
    import argparse
    parser = argparse.ArgumentParser(description='自动生成知识图谱关系')
    parser.add_argument('--head', required=True, help='头实体JSON文件路径')
    parser.add_argument('--tail', required=True, help='尾实体JSON文件路径')
    parser.add_argument('--relation', required=True, help='关系名称（如治疗）')
    parser.add_argument('--definition', required=True, help='关系定义（如该治疗方案用于治疗该疾病）')
    parser.add_argument('--output', required=True, help='输出文件路径')
    args = parser.parse_args()
    
    find_relations(args.head, args.tail, args.relation, args.definition, args.output)